## Preparation

In [4]:
import os,csv,random
import pandas as pd
import numpy as np
import scanpy as sc
import math

from skimage import io, color
import torch

from scanpy import read_10x_h5
import SpaGCN as spg
import matplotlib.pyplot as plt
import json

from tqdm import tqdm
import pickle

In [5]:
%cd /Users/anjideng1/Desktop/MALDI
import sys
sys.path.insert(0, '/Users/anjideng1/Desktop/MALDI/MALDI_package')
import MALDI_package as MALDIpy
MALDIpy.__version__

/Users/anjideng1/Desktop/MALDI


'1.0.0'

In [6]:
weekpoint1 = "5 wk regression"
weekpoint2 = "2 wk regression"

# compare within pos/neg group
group = "DHB pos" 
#group = "9AA neg"

DataDir = "data/Maaike"

In [7]:
MALDIdata1 = pd.read_csv("{DataDir}/{weekpoint} - {group} - All Spectra.csv".format(weekpoint = weekpoint1, DataDir = DataDir, group = group), sep =';') 
MALDIloc1 = pd.read_csv("{DataDir}/{weekpoint} - {group} - Region Spots.csv".format(weekpoint = weekpoint1, DataDir = DataDir, group = group), sep =';') 
MALDIdata2 = pd.read_csv("{DataDir}/{weekpoint} - {group} - All Spectra.csv".format(weekpoint = weekpoint2, DataDir = DataDir, group = group), sep =';') 
MALDIloc2 = pd.read_csv("{DataDir}/{weekpoint} - {group} - Region Spots.csv".format(weekpoint = weekpoint2, DataDir = DataDir, group = group), sep =';') 

In [8]:
mz_wk5 = pd.DataFrame(list(MALDIdata1.columns[1:]), index = list(MALDIdata1.columns[1:])).astype('float')
mz_wk5 = mz_wk5.rename(columns = {0:"m/z"})

mz_wk2 = pd.DataFrame(list(MALDIdata2.columns[1:]), index = list(MALDIdata2.columns[1:])).astype('float')
mz_wk2 = mz_wk2.rename(columns = {0:"m/z"})

In [9]:
# pandas.dataset.iloc(row, column) is used for retrive rows and columns from a dataset
MALDIdataAnn1 = sc.AnnData(X = MALDIdata1.iloc[:,1:], var = mz_wk5, obs = MALDIloc1) # number of rows does not match
MALDIdataAnn2 = sc.AnnData(X = MALDIdata2.iloc[:,1:], var = mz_wk2, obs = MALDIloc2)

/Users/anjideng1/anaconda3/lib/python3.10/site-packages/anndata/_core/anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/anjideng1/anaconda3/lib/python3.10/site-packages/anndata/_core/anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [10]:
MALDIdataAnn1.obs = MALDIdataAnn1.obs.astype(int)
sc.pp.normalize_per_cell(MALDIdataAnn1)

MALDIdataAnn2.obs = MALDIdataAnn2.obs.astype(int)
sc.pp.normalize_per_cell(MALDIdataAnn2)

## MSIWarp

In [11]:
import msiwarp as mx
from msiwarp.util.warp import to_mx_peaks
from msiwarp.util.warp import to_mz, to_height

In [37]:
def MSIwarping (MALDIdataAnn1, MALDIdataAnn2):
    spectra = []
    mzs = np.array(MALDIdataAnn1.var["m/z"]).astype(np.float) # peak m/z values
    meanspectrum1 =  np.mean(MALDIdataAnn1.X, axis = 0)
    for i in range(0,MALDIdataAnn1.X.shape[0],100): # 100?
        hs = meanspectrum1
        spectra.append( [mx.peak(i, mz_i, h_i, 1.0) for i, (mz_i, h_i) in enumerate(zip(mzs, hs))])
    #
    hs = meanspectrum1 # peak heights
    s = [mx.peak(i, mz_i, h_i, 1.0) for i, (mz_i, h_i) in enumerate(zip(mzs, hs))]
    
    #
    mzs2 = np.array(MALDIdataAnn2.var["m/z"]).astype(np.float) # peak m/z values
    meanspectrum2 =  np.mean(MALDIdataAnn2.X, axis = 0)
    hs2 = meanspectrum2 # peak heights
    s2 = [mx.peak(i, mz_i, h_i, 1.0) for i, (mz_i, h_i) in enumerate(zip(mzs2, hs2))]
    #
    reference_spectrum =  s2
    ### 1.2 setup the node placement parameters ######
    # method 1
    node_mzs = [i for i in range(15,1065,50)]
    node_deltas = [i*0.01 for i in range(15,121,5)] # [0.5] * len(node_mzs)# slacks = node_deltas * n_steps
    n_steps = 30
    nodes = mx.initialize_nodes(node_mzs, node_deltas, n_steps)
    epsilon = 1 # peak matching threshold, relative to peak width
    #
    optimal_moves = mx.find_optimal_spectra_warpings(spectra, reference_spectrum, nodes, epsilon)
    spectra2 = [[mx.peak(i, mz_i, h_i, 1.0) for i, (mz_i, h_i) in enumerate(zip(mzs.astype(np.float), mzs))]]
    #spectra2 = [[mx.peak(i, mz_i, h_i, 1.0) for i, (mz_i, h_i) in enumerate(zip(mzs.astype(np.float), hs))]]
    #
    warped_spectra = [mx.warp_peaks(s_i, nodes, optimal_moves[0]) for s_i in spectra2]
    #
    newmz = to_mz(warped_spectra[0])
    oldmz = to_height(warped_spectra[0])
    return {"newmz":newmz, "oldmz":oldmz}

In [36]:
node_mzs = [i for i in range(15,1065,50)]
# [0.5] * len(node_mzs)

[0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5]

In [32]:
# [i for i in range(15,121,5)]

[15,
 20,
 25,
 30,
 35,
 40,
 45,
 50,
 55,
 60,
 65,
 70,
 75,
 80,
 85,
 90,
 95,
 100,
 105,
 110,
 115,
 120]

In [38]:
warpresult = MSIwarping(MALDIdataAnn1, MALDIdataAnn2)

/var/folders/d7/m9p2b6151gq0xsqcd1hbfbgw0000gp/T/ipykernel_4023/2005656839.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mzs = np.array(MALDIdataAnn1.var["m/z"]).astype(np.float) # peak m/z values
/var/folders/d7/m9p2b6151gq0xsqcd1hbfbgw0000gp/T/ipykernel_4023/2005656839.py:13: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mzs2 = np

In [39]:
newint = warpresult.get("oldmz")
newint ## mean 

array([  15.05257511,   15.17888927,   15.30520344, ..., 1014.70452881,
       1014.83081055, 1014.95715332])

In [40]:
mean_int = np.mean(MALDIdataAnn1.X, axis = 0)
mean_int[40:50]

array([1.08559638, 1.0851067 , 1.08597486, 1.08470029, 1.08596298,
       1.08492318, 1.08561464, 1.08504546, 1.08584422, 1.08497938])

In [41]:
orgmz = warpresult.get("oldmz")

In [42]:
len(orgmz)

7917

In [43]:
newmz = warpresult.get("newmz")

In [44]:
newmz[50:60]

array([21.42041653, 21.54677279, 21.67312906, 21.79948533, 21.9258416 ,
       22.05219787, 22.17855413, 22.30491231, 22.43126858, 22.55762485])

In [45]:
MALDIdataAnn1.var[90:100]

,m/z
21.368293762207,21.368294
21.494607925415,21.494608
21.620922088623,21.620922
21.747236251831,21.747236
21.873550415039,21.873550
21.999864578247,21.999865
22.126178741455,22.126179
22.252494812012,22.252495
22.37880897522,22.378809
22.505123138428,22.505123


In [21]:
newmz

array([  15.11257511,   15.23888927,   15.36520344, ..., 1014.8190659 ,
       1014.94554548, 1015.07208619])

### Anchor point 1

In [22]:
orgmz[1420:1430]

array([194.41897583, 194.54528809, 194.6716156 , 194.79792786,
       194.92424011, 195.05055237, 195.17686462, 195.30317688,
       195.42948914, 195.55581665])

In [23]:
newmz[1420:1430]

array([194.4779952 , 194.60430324, 194.73062655, 194.85693459,
       194.98324264, 195.10955068, 195.23585873, 195.36216677,
       195.48847482, 195.61479812])

In [47]:
newmz[1420:1430] #

array([194.4856425 , 194.61195475, 194.73828227, 194.86459452,
       194.99090678, 195.11721903, 195.24353129, 195.36984355,
       195.4961558 , 195.62248332])

In [160]:
newmz

array([  15.11257511,   15.23888927,   15.36520344, ..., 1020.26799632,
       1020.39469479, 1020.52145449])

In [21]:
MALDIdataAnn1.var.iloc[2075:2090]

,m/z
272.10232543945,272.102325
272.22863769531,272.228638
272.35494995117,272.354950
272.48126220703,272.481262
272.60757446289,272.607574
272.73388671875,272.733887
272.86019897461,272.860199
272.98651123047,272.986511
273.11282348633,273.112823
273.23913574219,273.239136


### Anchor point 2

In [24]:
orgmz[2035:2050]

array([272.10232544, 272.2286377 , 272.35494995, 272.48126221,
       272.60757446, 272.73388672, 272.86019897, 272.98651123,
       273.11282349, 273.23913574, 273.365448  , 273.49179077,
       273.61810303, 273.74441528, 273.87072754])

In [25]:
newmz[2035:2050]

array([272.16804513, 272.29434054, 272.42063596, 272.54693137,
       272.67322679, 272.7995222 , 272.92581761, 273.05211303,
       273.17840844, 273.30470386, 273.43099927, 273.5573252 ,
       273.68362061, 273.80991603, 273.93621144])

In [48]:
newmz[2035:2050] 

array([272.16899211, 272.29530436, 272.42161662, 272.54792887,
       272.67424113, 272.80055339, 272.92686564, 273.0531779 ,
       273.17949015, 273.30580241, 273.43211466, 273.55845744,
       273.68476969, 273.81108195, 273.93739421])

### Anchor point 3

In [26]:
orgmz[5240:5250]

array([676.93988037, 677.06616211, 677.19250488, 677.31884766,
       677.44512939, 677.57147217, 677.69775391, 677.82409668,
       677.95037842, 678.07672119])

In [27]:
newmz[5240:5250] # matches

array([677.01430843, 677.14053123, 677.26681505, 677.39309886,
       677.51932167, 677.64560548, 677.77182829, 677.8981121 ,
       678.02433491, 678.15061872])

In [49]:
newmz[5240:5250] # matches

array([677.00654704, 677.13282878, 677.25917155, 677.38551432,
       677.51179606, 677.63813883, 677.76442057, 677.89076335,
       678.01704508, 678.14338786])

In [44]:
import numpy as np
import msiwarp as mx

In [45]:
# reference spectra
mzs_r = np.array(MALDIdataAnn2.var["m/z"]).astype(np.float) # peak m/z values
hs_r = np.mean(MALDIdataAnn2.X, axis = 0) # peak heights
s_r = [mx.peak(i, mz_i, h_i, 1.0) for i, (mz_i, h_i) in enumerate(zip(mzs_r, hs_r))]

/var/folders/d7/m9p2b6151gq0xsqcd1hbfbgw0000gp/T/ipykernel_779/647488041.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mzs_r = np.array(MALDIdataAnn2.var["m/z"]).astype(np.float) # peak m/z values


In [46]:
# unknown spectra
spectra = []
mzs = np.array(MALDIdataAnn1.var["m/z"]).astype(np.float) # peak m/z values
meanspectrum1 =  np.mean(MALDIdataAnn1.X, axis = 0)
for i in tqdm(range(0, MALDIdataAnn1.X.shape[0], 100)): # from 100 to 10 does not make a difference at anchor points
    hs = meanspectrum1
    spectra.append([mx.peak(i, mz_i, h_i, 1.0) for i, (mz_i, h_i) in enumerate(zip(mzs, hs))])

/var/folders/d7/m9p2b6151gq0xsqcd1hbfbgw0000gp/T/ipykernel_779/2396280870.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mzs = np.array(MALDIdataAnn1.var["m/z"]).astype(np.float) # peak m/z values
100%|██████████| 187/187 [00:02<00:00, 74.27it/s]


In [47]:
len(spectra)

187

In [48]:
### Setting up the warping nodes
node_mzs = [i for i in range(10,1030,102)]
node_deltas = [0.0001 * i for i in node_mzs] # slacks = node_deltas * n_steps
n_steps = 25
nodes = mx.initialize_nodes(node_mzs, node_deltas, n_steps)

epsilon = 1
n_cores = 4

In [49]:
optimal_moves = mx.find_optimal_spectra_warpings(spectra, s_r, nodes, epsilon)

In [50]:
spectra2 = [[mx.peak(i, mz_i, h_i, 1.0) for i, (mz_i, h_i) in enumerate(zip(mzs.astype(np.float), mzs))]]
warped_spectra = [mx.warp_peaks(s_i, nodes, o_i) for (s_i, o_i) in zip(spectra2, optimal_moves)]

/var/folders/d7/m9p2b6151gq0xsqcd1hbfbgw0000gp/T/ipykernel_779/2646325993.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  spectra2 = [[mx.peak(i, mz_i, h_i, 1.0) for i, (mz_i, h_i) in enumerate(zip(mzs.astype(np.float), mzs))]]


In [57]:
warped_spectra = [mx.warp_peaks(s_i, nodes, optimal_moves[0]) for s_i in spectra2]

In [58]:
newmz = to_mz(warped_spectra[0])
oldmz = to_height(warped_spectra[0])

In [38]:
newmz = []
newintensity = []

for s_i in zip(warped_spectra):
    newmz[i] = to_mz(s_i)
    newintensity[i] = to_height(s_i)

AttributeError: 'list' object has no attribute 'mz'

In [27]:
len(newmz)

7268

In [29]:
oldmz

array([1.08459845, 1.08459845, 1.08459845, ..., 1.08812968, 1.08761056,
       1.08838936])

In [ ]:
warped_spectra = [mx.warp_peaks(s_i, nodes, o_i) for (s_i, o_i) in zip(s, optimal_moves)]

In [80]:
MALDIdataAnn1.X

array([[1.06374088, 1.06374088, 1.06374088, ..., 1.07982464, 1.07982464,
        1.07982464],
       [1.06244145, 1.06244145, 1.06244145, ..., 1.09167818, 1.09167818,
        1.09167818],
       [1.05965938, 1.05965938, 1.05965938, ..., 1.05260602, 1.05260602,
        1.05260602],
       ...,
       [1.09549556, 1.09549556, 1.09549556, ..., 1.09467593, 1.09467593,
        1.09467593],
       [1.0892313 , 1.0892313 , 1.0892313 , ..., 1.09895997, 1.09895997,
        1.09895997],
       [1.08235046, 1.08235046, 1.08235046, ..., 1.08845674, 1.08845674,
        1.08845674]])

In [59]:
meanspectrum_aligned = np.mean(MALDIdataAnn1.X, axis = 0)
meanspectrum_aligned_mv = np.convolve(meanspectrum_aligned, np.ones(3)/3, mode='valid')

### output the result

In [58]:
mz_aligned = MALDIdataAnn1.var

In [59]:
np.array(mz_aligned["m/z"][0:40])

array([10.        , 10.12631416, 10.25262833, 10.37894344, 10.50525761,
       10.63157177, 10.75788593, 10.88420105, 11.01051521, 11.13682938,
       11.26314354, 11.3894577 , 11.51577282, 11.64208698, 11.76840115,
       11.89471531, 12.02102947, 12.14734459, 12.27365875, 12.39997292,
       12.52628708, 12.6526022 , 12.77891636, 12.90523052, 13.03154469,
       13.15785885, 13.28417397, 13.41048813, 13.53680229, 13.66311646,
       13.78943157, 13.91574574, 14.0420599 , 14.16837406, 14.29468822,
       14.42100334, 14.5473175 , 14.67363167, 14.79994583, 14.92625999])

In [60]:
newmz[:-1]

array([  15.10259264,   15.2289489 ,   15.35530517, ..., 1014.67776422,
       1014.80423334, 1014.93064136])

In [63]:
len(newmz)

7917

In [61]:
newmz_all = np.concatenate((np.array(mz_aligned["m/z"][1:40]), newmz[:-1]))

In [62]:
len(newmz_all)

7955

In [65]:
newmz_all

array([  10.12631416,   10.25262833,   10.37894344, ..., 1020.14123661,
       1020.26799632, 1020.39469479])

In [66]:
np.array(mz_aligned["m/z"][0:40])

array([10.        , 10.12631416, 10.25262833, 10.37894344, 10.50525761,
       10.63157177, 10.75788593, 10.88420105, 11.01051521, 11.13682938,
       11.26314354, 11.3894577 , 11.51577282, 11.64208698, 11.76840115,
       11.89471531, 12.02102947, 12.14734459, 12.27365875, 12.39997292,
       12.52628708, 12.6526022 , 12.77891636, 12.90523052, 13.03154469,
       13.15785885, 13.28417397, 13.41048813, 13.53680229, 13.66311646,
       13.78943157, 13.91574574, 14.0420599 , 14.16837406, 14.29468822,
       14.42100334, 14.5473175 , 14.67363167, 14.79994583, 14.92625999])

In [67]:
newmz_all[100:110]

array([22.81775146, 22.94406563, 23.07037979, 23.19669395, 23.32300812,
       23.44932419, 23.57563835, 23.70195251, 23.82826668, 23.95458084])

In [68]:
mz_aligned["m/z"][100:110]

22.631437301636    22.631437
22.757751464844    22.757751
22.884065628052    22.884066
23.01037979126     23.010380
23.136693954468    23.136694
23.263008117676    23.263008
23.389324188232    23.389324
23.51563835144     23.515638
23.641952514648    23.641953
23.768266677856    23.768267
Name: m/z, dtype: float64

In [69]:
len(meanspectrum_aligned_mv)

7998

In [70]:
originaltable= pd.DataFrame(newmz_all)
originaltable = originaltable.rename(columns = {0:"mz_aligned_msi"})
originaltable["intensity_msi"] = meanspectrum_aligned_mv
originaltable

,mz_aligned_msi,intensity_msi
0,10.126314,1.084598
1,10.252628,1.084330
2,10.378943,1.084302
3,10.505258,1.084037
4,10.631572,1.084169
...,...,...
7993,1019.887778,1.089992
7994,1020.014538,1.088412
7995,1020.141237,1.088537
7996,1020.267996,1.088109


In [71]:
csv_file_path = '/Users/anjideng1/Desktop/MALDI/output_Maaike'
originaltable.to_csv (csv_file_path + "/mean_spec_mv_msi.csv")